# DLS Final Project - Modeling school test scores using satellite images

In [2]:
import numpy as np
from PIL import Image
import pandas as pd
import numpy as np
import glob
import time
import winsound

In [3]:
# !pip install numpy==1.18.5 
# import numpy
# np.__version__

In [4]:
import tensorflow as tf


## Read in school data to get successes. Sort data by ncesid

In [5]:
school_df = pd.read_csv("C:\\Users\\lvden\\OneDrive - Montana State University\\Desktop\\DeepLearning\\final_project\\cleaned_school_subset.csv")

In [6]:
### Get the average passing proportion and then make a column with 1 being >= the proportion and 0 being <
mean_pass_prop = np.mean(school_df['ALL_MTH00PCTPROF_1415'])

above_or_below_list = []
for val in school_df['ALL_MTH00PCTPROF_1415']:
    if val >= mean_pass_prop:
        above_or_below_list.append(1)
    else:
        above_or_below_list.append(0)

school_df['above_or_below'] = above_or_below_list

In [7]:
### Sorting by ncesid so that the schools align with their images, which are labeled and sorted by ncesid as well
school_sorted_df = school_df.sort_values(by='NCESID')

In [8]:
school_scores_aob = school_sorted_df['above_or_below'].values

In [9]:
good_ncesid = school_sorted_df['NCESID'].values

## Read in images function

In [10]:
def image_read_and_processing(type, input_shape=224):
    '''Function for reading in and cleaning data.
        type: 1 for small images, 2 for large images, 3 for combined images.
        input_shape: int for square size of images.'''
    
    if (type == 1) or (type==3):
        zoomed_in_paths = glob.glob("C:\\Users\\lvden\\OneDrive - Montana State University\\Desktop\\DeepLearning\\final_project\\img_data-20231201T222346Z-001\\img_data_screenshots_cali_high_100\\small\\*")
    if (type == 2) or (type==3):
        zoomed_out_paths = glob.glob("C:\\Users\\lvden\\OneDrive - Montana State University\\Desktop\\DeepLearning\\final_project\\img_data-20231201T222346Z-001\\img_data_screenshots_cali_high_100\\large\\*")

    good_nc_list = []
    ### Getting the zoomed in images that match the school dataset
    if (type == 1) or (type==3):
        zoomed_in_paths_clean_list = []
        for a in zoomed_in_paths:
            if int(a.split('\\')[-1].split('_')[-1].split('.')[0]) in good_ncesid:
                zoomed_in_paths_clean_list.append(a)
                if type == 1:
                    good_nc_list.append(int(a.split('\\')[-1].split('_')[-1].split('.')[0]))

    ### Getting the zoomed out images that match the school dataset
    if (type == 2) or (type==3):
        zoomed_out_paths_clean_list = []
        for a in zoomed_out_paths:
            if int(a.split('\\')[-1].split('_')[-1].split('.')[0]) in good_ncesid:
                zoomed_out_paths_clean_list.append(a)    
                if (type == 2) or (type==3):
                    good_nc_list.append(int(a.split('\\')[-1].split('_')[-1].split('.')[0]))


    ### Subseting the dataframe to include only the values needed for the images
    school_sorted_subset_df = pd.DataFrame()
    for row in school_sorted_df.iterrows():
        if row[1]['NCESID'] in good_nc_list:
            school_sorted_subset_df = school_sorted_subset_df.append(row[1])
    
    
    ### Opening the image and converting to an array. Then adding the arrays into a list to cast to an array
    if (type == 1) or (type==3):
        zoomed_in_arrays_list = []
        for path in zoomed_in_paths_clean_list:
            img_array = np.asanyarray(Image.open(path))
            zoomed_in_arrays_list.append(img_array)
        
        zoomed_in_array = np.asanyarray(zoomed_in_arrays_list)
        del(zoomed_in_arrays_list)

    
    ### Opening the image and converting to an array. Then adding the arrays into a list to cast to an array
    if (type == 2) or (type==3):
        zoomed_out_arrays_list = []
        for path in zoomed_out_paths_clean_list:
            img_array = np.asanyarray(Image.open(path))
            zoomed_out_arrays_list.append(img_array)
        
        zoomed_out_array = np.asanyarray(zoomed_out_arrays_list)
        del(zoomed_out_arrays_list)


    ### Getting the combined zoomed in and zoomed out images
    if type == 3:
        i = 0
        combined_img_array_list = []
        while i < len(zoomed_in_array):
            comb = np.concatenate([zoomed_in_array[i], zoomed_out_array[i]], axis=0)
            combined_img_array_list.append(comb)
            i+=1
        
        combined_array = np.asanyarray(combined_img_array_list)
        del(combined_img_array_list)
        del(zoomed_in_array)
        del(zoomed_out_array)

    if type == 1:
        zoomed_in_array_reshaped = tf.image.resize_with_crop_or_pad(zoomed_in_array, input_shape,input_shape)
    if type == 2:
        zoomed_out_array_reshaped = tf.image.resize_with_crop_or_pad(zoomed_out_array, input_shape,input_shape)
    if type == 3:
        combined_array_reshaped = tf.image.resize_with_crop_or_pad(combined_array, input_shape,input_shape)

    if type == 1:
        zoomed_in_array_rescaled_reshaped = zoomed_in_array_reshaped/255
    if type == 2:
        zoomed_out_array_rescaled_reshaped = zoomed_out_array_reshaped/255
    if type == 3:
        combined_array_rescaled_reshaped = combined_array_reshaped/255


    ### Partitioning to 80% train, 20% validation
    if type == 1:
        zoomed_in_dataset_train = tf.data.Dataset.from_tensors((zoomed_in_array_rescaled_reshaped[0:724],  school_scores_aob[0:724]))
        zoomed_in_dataset_val = tf.data.Dataset.from_tensors((zoomed_in_array_rescaled_reshaped[724:815], school_scores_aob[724:815]))
        zoomed_in_dataset_test = tf.data.Dataset.from_tensors(zoomed_in_array_rescaled_reshaped[815:], school_scores_aob[815:])
        return zoomed_in_dataset_train, zoomed_in_dataset_val, zoomed_in_dataset_test

    if type == 2:
        # zoomed_out_dataset_train = tf.data.Dataset.from_tensors((zoomed_out_array_rescaled_reshaped[0:724],  school_scores_aob[0:724]))
        # zoomed_out_dataset_val = tf.data.Dataset.from_tensors((zoomed_out_array_rescaled_reshaped[724:815], school_scores_aob[724:815]))
        # zoomed_out_dataset_test = tf.data.Dataset.from_tensors(zoomed_out_array_rescaled_reshaped[815:])
        # return zoomed_out_dataset_train, zoomed_out_dataset_val, zoomed_out_dataset_test
        
        zoomed_out_dataset_train = zoomed_out_array_rescaled_reshaped[0:724]
        label_train = tf.convert_to_tensor(school_sorted_subset_df['above_or_below'][0:724])
        zoomed_out_dataset_val = zoomed_out_array_rescaled_reshaped[724:]
        label_val = tf.convert_to_tensor(school_sorted_subset_df['above_or_below'][724:])
        return zoomed_out_dataset_train, zoomed_out_dataset_val, label_train, label_val

    if type == 3:
        # combined_dataset_train = tf.data.Dataset.from_tensors((combined_array_rescaled_reshaped[0:724], school_scores_aob[0:724]))
        # combined_dataset_val = tf.data.Dataset.from_tensors((combined_array_rescaled_reshaped[724:815], school_scores_aob[724:815]))
        # combined_dataset_test = tf.data.Dataset.from_tensors((combined_array_rescaled_reshaped[815:], school_scores_aob[815:]))
        # return combined_dataset_train, combined_dataset_val, combined_dataset_test
           
        combined_dataset_train = combined_array_rescaled_reshaped[0:724]
        label_train = tf.convert_to_tensor(school_sorted_subset_df['above_or_below'][0:724])
        combined_dataset_val = combined_array_rescaled_reshaped[724:]
        label_val = tf.convert_to_tensor(school_sorted_subset_df['above_or_below'][724:])
        return combined_dataset_train, combined_dataset_val, label_train, label_val


In [11]:
#zoom_out_train, zoom_out_val, zoom_out_test, label_train, label_val, label_test = image_read_and_processing(type=2, input_shape=224)
combined_train, combined_val, label_train, label_val = image_read_and_processing(type=3, input_shape=224)

C:\Users\lvden\AppData\Local\Temp\ipykernel_15324\6474798.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  school_sorted_subset_df = school_sorted_subset_df.append(row[1])
C:\Users\lvden\AppData\Local\Temp\ipykernel_15324\6474798.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  school_sorted_subset_df = school_sorted_subset_df.append(row[1])
C:\Users\lvden\AppData\Local\Temp\ipykernel_15324\6474798.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  school_sorted_subset_df = school_sorted_subset_df.append(row[1])
C:\Users\lvden\AppData\Local\Temp\ipykernel_15324\6474798.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  schoo

## Modeling

- Increasing image size did not help model accuracy.
- Image augmentation did not help with model accuracy.
- These models were run using the same code and changing the data imported since I would overload my disk space trying to keep both datasets

### resnet example

In [25]:
resnet = tf.keras.applications.ResNet50V2(include_top = True, weights=None, classes=2)

In [26]:
resnet.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=1e-4),
               loss = tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [27]:
train_history = resnet.fit(x=combined_train,y=label_train, validation_data = (combined_val, label_val), epochs=30, 
                           batch_size=8, validation_batch_size=8)

Epoch 1/30
91/91 [==============================] - 9s 97ms/step - loss: 0.7067 - accuracy: 0.4765 - val_loss: 0.6934 - val_accuracy: 0.5769
Epoch 2/30
91/91 [==============================] - 8s 90ms/step - loss: 0.6953 - accuracy: 0.5207 - val_loss: 0.6940 - val_accuracy: 0.4231
Epoch 3/30
91/91 [==============================] - 8s 90ms/step - loss: 0.6947 - accuracy: 0.5290 - val_loss: 0.7090 - val_accuracy: 0.4231
Epoch 4/30
91/91 [==============================] - 8s 90ms/step - loss: 0.6943 - accuracy: 0.5207 - val_loss: 0.6939 - val_accuracy: 0.5714
Epoch 5/30
91/91 [==============================] - 8s 90ms/step - loss: 0.6939 - accuracy: 0.5028 - val_loss: 0.6940 - val_accuracy: 0.5879
Epoch 6/30
91/91 [==============================] - 8s 91ms/step - loss: 0.6937 - accuracy: 0.4931 - val_loss: 0.6948 - val_accuracy: 0.5769
Epoch 7/30
91/91 [==============================] - 8s 91ms/step - loss: 0.6935 - accuracy: 0.5097 - val_loss: 0.6943 - val_accuracy: 0.5769
Epoch 8/30
91

In [24]:
winsound.Beep(350,350)

In [28]:
resnet.save('combined_6209_val_acc')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: combined_6209_val_acc\assets


## Image augmentation

In [111]:
import imgaug as ia
import imgaug.augmenters as iaa
import imageio

In [133]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=40)

In [134]:
zoom_out_train_aug = train_gen.flow(zoom_out_train,y=label_train, batch_size=16)


In [135]:
resnet = tf.keras.applications.ResNet50V2(include_top = True, weights=None, classes=2)

In [136]:
resnet.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=1e-3),
               loss = tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [138]:
train_history = resnet.fit(x=zoom_out_train_aug, validation_data = (zoom_out_val, label_val), epochs=30, 
                           batch_size=16, validation_batch_size=16)

Epoch 1/30


ResourceExhaustedError:  OOM when allocating tensor with shape[16,512,28,28] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node resnet50v2/conv3_block3_3_conv/Conv2D (defined at \AppData\Local\Temp\ipykernel_10336\868025829.py:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_94762]

Function call stack:
train_function


In [ ]:
winsound.Beep(350,350)

# Extras from testing 

## Read in images

In [21]:
zoomed_in_paths = glob.glob("C:\\Users\\lvden\\OneDrive - Montana State University\\Desktop\\DeepLearning\\final_project\\img_data-20231201T222346Z-001\\img_data_screenshots_cali_high_100\\small\\*")
print(len(zoomed_in_paths))


906


In [22]:
zoomed_out_paths = glob.glob("C:\\Users\\lvden\\OneDrive - Montana State University\\Desktop\\DeepLearning\\final_project\\img_data-20231201T222346Z-001\\img_data_screenshots_cali_high_100\\large\\*")
print(len(zoomed_out_paths))


906


## Number of images and length of school dataset does not match, need to only have the images in the dataset

In [24]:
### Getting the zoomed in images that match the school dataset
zoomed_in_paths_clean_list = []
for a in zoomed_in_paths:
    if int(a.split('\\')[-1].split('_')[-1].split('.')[0]) in good_ncesid:
        zoomed_in_paths_clean_list.append(a)

print(len(zoomed_in_paths_clean_list))

906


In [25]:
### Getting the zoomed out images that match the school dataset
zoomed_out_paths_clean_list = []
for a in zoomed_out_paths:
    if int(a.split('\\')[-1].split('_')[-1].split('.')[0]) in good_ncesid:
        zoomed_out_paths_clean_list.append(a)

print(len(zoomed_out_paths_clean_list))

906


## Making images into arrays 

In [26]:
### Opening the image and converting to an array. Then adding the arrays into a list to cast to an array
start = time.time()
zoomed_in_arrays_list = []
for path in zoomed_in_paths_clean_list:
    img_array = np.asanyarray(Image.open(path))
    zoomed_in_arrays_list.append(img_array)

zoomed_in_array = np.asanyarray(zoomed_in_arrays_list)
del(zoomed_in_arrays_list)
end = time.time()
print(end-start)

158.81427526474


In [27]:
### Opening the image and converting to an array. Then adding the arrays into a list to cast to an array
start = time.time()
zoomed_out_arrays_list = []
for path in zoomed_out_paths_clean_list:
    img_array = np.asanyarray(Image.open(path))
    zoomed_out_arrays_list.append(img_array)

zoomed_out_array = np.asanyarray(zoomed_out_arrays_list)
del(zoomed_out_arrays_list)
end = time.time()
print(end-start)

104.10796523094177


In [28]:
print(zoomed_in_array.shape)
print(zoomed_out_array.shape)

(906, 755, 1890, 3)
(906, 755, 1890, 3)


In [29]:
### Getting the combined zoomed in and zoomed out images

i = 0
combined_img_array_list = []
while i < len(zoomed_in_array):
    comb = np.concatenate([zoomed_in_array[i], zoomed_out_array[i]], axis=0)
    combined_img_array_list.append(comb)
    i+=1

combined_array = np.asanyarray(combined_img_array_list)
del(combined_img_array_list)


## Image preprocessing

In [30]:
zoomed_in_array_reshaped = tf.image.resize_with_crop_or_pad(zoomed_in_array, 224,224)
zoomed_out_array_reshaped = tf.image.resize_with_crop_or_pad(zoomed_out_array, 224,224)
combined_array_reshaped = tf.image.resize_with_crop_or_pad(combined_array, 224,224)

In [31]:
zoomed_in_array_rescaled_reshaped = zoomed_in_array_reshaped/255
zoomed_out_array_rescaled_reshaped = zoomed_out_array_reshaped/255
combined_array_rescaled_reshaped = combined_array_reshaped/255

## Putting data into dataset and partitioning into train/validation/test

In [32]:
# ### Partitioning to 80% train, 10% validation, and 10% test
# zoomed_in_dataset_train = tf.data.Dataset.from_tensors(zoomed_in_array[0:724])
# zoomed_in_dataset_train.batch(32)
# zoomed_in_dataset_val = tf.data.Dataset.from_tensors(zoomed_in_array[724:815])
# zoomed_in_dataset_val.batch(32)
# zoomed_in_dataset_test = tf.data.Dataset.from_tensors(zoomed_in_array[815:])
# zoomed_in_dataset_test.batch(32)

# zoomed_out_dataset_train = tf.data.Dataset.from_tensors(zoomed_out_array[0:724])
# zoomed_out_dataset_train.batch(32)
# zoomed_out_dataset_val = tf.data.Dataset.from_tensors(zoomed_out_array[724:815])
# zoomed_out_dataset_val.batch(32)
# zoomed_out_dataset_test = tf.data.Dataset.from_tensors(zoomed_out_array[815:])
# zoomed_out_dataset_test.batch(32)

# combined_dataset_train = tf.data.Dataset.from_tensors(combined_array[0:724])
# combined_dataset_train.batch(32)
# combined_dataset_val = tf.data.Dataset.from_tensors(combined_array[724:815])
# combined_dataset_val.batch(32)
# combined_dataset_test = tf.data.Dataset.from_tensors(combined_array[815:])
# combined_dataset_test.batch(32)

In [33]:
# school_scores_aob_train = school_scores_aob[0:724]
# school_scores_aob_val = school_scores_aob[724:815]
# school_scores_aob_test = school_scores_aob[815:]

In [34]:
### Partitioning to 80% train, 10% validation, and 10% test
zoomed_in_dataset_train = tf.data.Dataset.from_tensors((zoomed_in_array_rescaled_reshaped[0:724],  school_scores_aob[0:724]))
zoomed_in_dataset_val = tf.data.Dataset.from_tensors((zoomed_in_array_rescaled_reshaped[724:815], school_scores_aob[724:815]))
zoomed_in_dataset_test = tf.data.Dataset.from_tensors(zoomed_in_array_rescaled_reshaped[815:])

zoomed_out_dataset_train = tf.data.Dataset.from_tensors((zoomed_out_array_rescaled_reshaped[0:724],  school_scores_aob[0:724]))
zoomed_out_dataset_val = tf.data.Dataset.from_tensors((zoomed_out_array_rescaled_reshaped[724:815], school_scores_aob[724:815]))
zoomed_out_dataset_test = tf.data.Dataset.from_tensors(zoomed_out_array_rescaled_reshaped[815:])

combined_dataset_train = tf.data.Dataset.from_tensors((combined_array_rescaled_reshaped[0:724], school_scores_aob[0:724]))
combined_dataset_val = tf.data.Dataset.from_tensors((combined_array_rescaled_reshaped[724:815], school_scores_aob[724:815]))
combined_dataset_test = tf.data.Dataset.from_tensors((combined_array_rescaled_reshaped[815:], school_scores_aob[724:815]))


In [35]:
zoomed_in_dataset_train

<TensorDataset shapes: ((724, 224, 224, 3), (724,)), types: (tf.float32, tf.int64)>